In [1]:
# For getting file general DTS pathes
from hibachiPostgreSQL import ReadAllGeneralDTSPathesInPostgre 
import pandas as pd
# Get all the general DTS file pathes saved in PostgreSQL 
df_DTSPathes = ReadAllGeneralDTSPathesInPostgre.generalDTSPathes()

Connecting to the PostgreSQL database...
Connection is closed.


In [2]:
# To verify if there are duplicated files,
# Separate the file name from the path and put it into dataframe
import os 
df_DTSPathes['filename'] = df_DTSPathes['filepath'].apply(lambda x: os.path.split(x)[1])

In [3]:
df_DTSPathes.head()

,filepath,filename
0,EdinetGeneraltaxonomy\common\2013-08-31/identi...,identificationAndOrdering_2013-08-31.xsd
1,EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_c...,jpcrp_cor_2013-08-31.xsd
2,EdinetGeneraltaxonomy\jpcrp\2013-08-31/jpcrp_r...,jpcrp_rt_2013-08-31.xsd
3,EdinetGeneraltaxonomy\jpcrp\2013-08-31\label/j...,jpcrp_2013-08-31_gla.xml
4,EdinetGeneraltaxonomy\jpcrp\2013-08-31\label/j...,jpcrp_2013-08-31_lab-en.xml


In [4]:
# List all the pathes with the same file names
df_duplicatedDTS=df_DTSPathes[(df_DTSPathes.duplicated(subset=['filename'])) == True]

In [5]:
print('Following files have the same file names.')
df_duplicatedDTS

Following files have the same file names.


,filepath,filename
10026,EdinetGeneraltaxonomy\jppfs\2015-03-31/jppfs_p...,jppfs_pe_2012.xsd
12131,EdinetGeneraltaxonomy\jppfs\2016-02-29/jppfs_p...,jppfs_pe_2012.xsd
14238,EdinetGeneraltaxonomy\jppfs\2017-02-28/jppfs_p...,jppfs_pe_2012.xsd
16343,EdinetGeneraltaxonomy\jppfs\2018-02-28/jppfs_p...,jppfs_pe_2012.xsd
18449,EdinetGeneraltaxonomy\jppfs\2019-02-28/jppfs_p...,jppfs_pe_2012.xsd
20555,EdinetGeneraltaxonomy\jppfs\2019-11-01/jppfs_p...,jppfs_pe_2012.xsd


In [10]:
import filecmp
for firstIndex in range(0,df_duplicatedDTS['filename'].count()-1):
    firstFilePath = df_duplicatedDTS.iloc[firstIndex]['filepath']
    for secondIndex in range(0+1,df_duplicatedDTS['filename'].count()):
        secondFilePath = df_duplicatedDTS.iloc[secondIndex]['filepath']
        # If the file names are the same, compare the contents inside        
        if df_duplicatedDTS.iloc[firstIndex]['filename'] == df_duplicatedDTS.iloc[secondIndex]['filename']:
            if filecmp.cmp(firstFilePath,secondFilePath) == True:
                print('Pass: 2 files have the same contents.')
                #print('For:',firstFilePath,' & ',secondFilePath)
            else:
                print('File names are the same but the contents are different.')
                print('For:',firstFilePath,' & ',secondFilePath)

Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.
Pass: 2 files have the same contents.


In [11]:
df_duplicatedDTS['content']='no duplicate'

<ipython-input-11-20200c00e03e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_duplicatedDTS['content']='no duplicate'


In [13]:
# Save the dataframe containing the duplicate general DTS to PostgreSQL
# Connect to PostgreSQL
from sqlalchemy import create_engine
from config import config

try:
    params = config()
    engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(**params))

    # Save the dataframe to PostgreSQL (make sure to type table names in lower cases)
    df_duplicatedDTS.to_sql('taxonomyschema_generaldts_duplicates', con=engine, if_exists='replace', index=False)
except:
    print ('Error:', sys.exc_info()[0], sys.exc_info()[1])